In [ ]:
from research.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
import cv2

import json
import os
import  pandas as pd
import matplotlib.patches as patches
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [ ]:
s3_access_utils = S3AccessUtils('/root/data',
                               json.load(open(os.environ['AWS_CREDENTIALS'])))

In [ ]:
bolaks_data = pd.read_csv(open("/root/data/jane/bolaks_poc.csv"))
matches = pd.read_csv(open("/root/data/jane/Fish_ID_Annotation.csv"))

In [ ]:
bolaks_data.shape

In [ ]:
bolaks_data.head(3)

In [ ]:
matches.dtypes

In [ ]:
matches.loc[~matches.iloc[:, 3].notnull()]

In [ ]:
#pairs = matches.loc[(matches.iloc[:, 1] > 0) & (~matches.iloc[:, 3].notnull()) ].iloc[:, 0]
pairs = matches.loc[(matches.iloc[:, 1] > 0) & (matches.iloc[:, 2] != "0")].iloc[:, 0]
print("1-indexed pair")
pairs


In [ ]:
print('get 0-indexed pair')
p1 = pairs.apply(lambda x: int(x.split('_matches_')[0]) - 1)
p2 = pairs.apply(lambda x: int(x.split('_matches_')[1].split('.jpg')[0]) - 1)
pairs_data = pd.DataFrame({'p1': p1,'p2':p2}).reset_index().drop(columns=['index'])


pairs_data["p1_w"] = bolaks_data.iloc[pairs_data["p1"]]["estimated_weight_g"].reset_index().drop(columns=['index'])

pairs_data["p2_w"] = bolaks_data.iloc[pairs_data["p2"]]["estimated_weight_g"].reset_index().drop(columns=['index'])


pairs_data["p1_left_crop_url"] = bolaks_data.iloc[pairs_data["p1"]]["left_crop_url"].reset_index().drop(columns=['index'])
pairs_data["p1_right_crop_url"] = bolaks_data.iloc[pairs_data["p1"]]["right_crop_url"].reset_index().drop(columns=['index'])
pairs_data["p2_left_crop_url"] = bolaks_data.iloc[pairs_data["p2"]]["left_crop_url"].reset_index().drop(columns=['index'])
pairs_data["p2_right_crop_url"] = bolaks_data.iloc[pairs_data["p2"]]["right_crop_url"].reset_index().drop(columns=['index'])


pairs_data["diff"] = np.abs(pairs_data["p1_w"] - pairs_data["p2_w"])
pairs_data['min_w'] = np.minimum(pairs_data["p1_w"], pairs_data["p2_w"])
pairs_data['max_w'] = np.maximum(pairs_data["p1_w"], pairs_data["p2_w"])

pairs_data["p1_captured_at"] = bolaks_data.iloc[pairs_data["p1"]]["captured_at"].reset_index().drop(columns=['index'])
pairs_data["p2_captured_at"] = bolaks_data.iloc[pairs_data["p1"]]["captured_at"].reset_index().drop(columns=['index'])


pairs_data["p1_group_id"] = bolaks_data.iloc[pairs_data["p1"]]["group_id"].reset_index().drop(columns=['index'])
pairs_data["p2_group_id"] = bolaks_data.iloc[pairs_data["p1"]]["group_id"].reset_index().drop(columns=['index'])

pairs_data



In [ ]:
def plot_image(url):

    image_f, bucket, image_key = s3_access_utils.download_from_url(url)

    img = Image.open(image_f)

    alpha = 2 # Contrast control (1.0-3.0)
    beta = 20 # Brightness control (0-100)

    img = np.asarray(img)
    adjusted = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    return adjusted

In [ ]:
bolaks_data.iloc[11818].left_crop_url

In [ ]:
plt.imshow(plot_image(bolaks_data.iloc[11818].left_crop_url))

In [ ]:
num_img = 0
for idx, row in tqdm(pairs_data.loc[(pairs_data["diff"] > 2000) & (pairs_data["diff"] <= 8000)].iterrows()):
    #if idx < 000: continue
    num_img += 1
    #if num_img > 20: break
    figure, axes = plt.subplots(nrows=1, ncols=2,figsize=(15,15))
    
    axes[0].imshow(plot_image(row.p1_left_crop_url))
    axes[0].set_title("predicted_weight {} idx{}".format(row.p1_w, row.p1))
    
    axes[1].imshow(plot_image(row.p2_left_crop_url))
    axes[1].set_title("predicted_weight {} idx{}".format(row.p2_w, row.p2))
print(num_img)

In [ ]:
pairs_data['diff'].describe()

In [ ]:

plt.plot(pairs_data['min_w'], pairs_data['diff'], '.')
plt.title("abs errror distribution")
plt.xlabel("min weight of paired fish (g)")
plt.ylabel("abs error (g)")

In [ ]:
plt.hist(pairs_data['diff'], bins = 20)
plt.title("histogram of abs errror")
plt.xlabel("count")
plt.ylabel("abs error (g)")

In [ ]:
def get_abs_error_pct(y_pred, y_gt):
    return np.mean(np.abs((y_pred - y_gt) / y_gt))

def get_pct_diff(y_pred, y_gt):
    return (np.mean(y_pred) - np.mean(y_gt)) / np.mean(y_gt)


In [ ]:
get_abs_error_pct(pairs_data['p1_w'], pairs_data['p2_w'])

In [ ]:
get_pct_diff(pairs_data['max_w'], pairs_data['min_w'])

In [ ]:
get_pct_diff(pairs_data['p1_w'], pairs_data['p2_w'])

In [ ]:
(np.abs((pairs_data['p1_w'] - pairs_data['p2_w']) / pairs_data['p2_w'])).describe()

In [ ]:
output_bucket='aquabyte-images-adhoc' 
output_path = "bolaks_data_pairs.csv"
pairs_data.to_csv(output_path, index=False)
output_key = os.path.join("jane/fish_id", output_path)
s3_access_utils.s3_client.upload_file(output_path, output_bucket, output_key)